# Figures for the Plos CB paper that do not use data from evolutionary runs

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import pickle
import itertools

In [ ]:
import importlib
import evotsc
import evotsc_run
import evotsc_plot
importlib.reload(evotsc)
importlib.reload(evotsc_run)
importlib.reload(evotsc_plot)

In [ ]:
## Constants
# Population
nb_indivs = 100
nb_genes = 60

# Genome
intergene = 125
gene_length = 1000
interaction_dist = 5000
interaction_coef = 0.03
sigma_basal = -0.066
sigma_opt = -0.042
epsilon = 0.005
m = 2.5
default_basal_expression = (1 + np.exp(- m)) / 2 # Average of the maximum and minimum expression levels in the model

# Fitness
selection_coef = 50

# Selection
selection_method = "fit-prop" # Choices: "fit-prop", "rank", "exp-rank"

# Environment
sigma_A = 0.01
sigma_B = -0.01

# Mutations
inversion_poisson_lam = 2.0
intergene_poisson_lam = 0.0 #2.0
intergene_mutation_var = 0.0 #1e1
basal_sc_mutation_prob = 0.0 #1e-1
basal_sc_mutation_var = 0.0 #1e-4

In [ ]:
gene_types = ['AB', 'A', 'B'] # Name of each gene type
gene_type_color = ['tab:blue', 'tab:red', 'tab:green'] #AB, A, B
dpi = 300

In [ ]:
rng = np.random.default_rng(seed=123456)

# Helper functions

In [ ]:
def plot_expr(indiv, sigma_env, plot_title=None, plot_name=None):
    
    if not indiv.already_evaluated:
        indiv.evaluate(sigma_env, sigma_env)
    
    # Plot only environment A
    temporal_expr = indiv.run_system(sigma_env)

    nb_genes, nb_steps = temporal_expr.shape

    colors = mpl.cm.get_cmap('viridis', indiv.nb_genes)(range(indiv.nb_genes))

    plt.figure(figsize=(8, 5), dpi=dpi)

    plt.ylim(-0.05, 1.05)

    for gene in range(indiv.nb_genes):
        linestyle = 'solid' if indiv.genes[gene].orientation == 0 else 'dashed'
        plt.plot(temporal_expr[:, indiv.genes[gene].id],
                 linestyle=linestyle,
                 linewidth=2,
                 color=colors[indiv.genes[gene].id],
                 label=f'Gene {indiv.genes[gene].id}')

    plt.grid(linestyle=':')
    plt.xlabel('Iteration steps', fontsize='large')
    plt.ylabel('Gene expression level', fontsize='large')
    
    #plt.legend(ncol=2)

    if plot_title:
        plt.title(plot_title)

    plt.tight_layout()
    if plot_name:
        plt.savefig(plot_name + '.pdf', dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()


In [ ]:
def make_random_indiv(nb_genes=nb_genes, default_basal_expression=None, nb_mut=0):
    genes = evotsc.Gene.generate(intergene=intergene,
                                 length=gene_length,
                                 nb_genes=nb_genes,
                                 default_basal_expression=default_basal_expression,
                                 rng=rng)
    
    indiv = evotsc.Individual(genes=genes,
                              interaction_dist=interaction_dist,
                              interaction_coef=interaction_coef,
                              sigma_basal=sigma_basal,
                              sigma_opt=sigma_opt,
                              epsilon=epsilon,
                              m=m,
                              selection_coef=selection_coef,
                              rng=rng)
    
    mutation = evotsc.Mutation(basal_sc_mutation_prob=basal_sc_mutation_prob,
                               basal_sc_mutation_var=basal_sc_mutation_var,
                               intergene_poisson_lam=intergene_poisson_lam,
                               intergene_mutation_var=intergene_mutation_var,
                               inversion_poisson_lam=inversion_poisson_lam)
    
    for i_mut in range(nb_mut):
        indiv.mutate(mutation)
    
    return indiv

In [ ]:
def compute_extended_phenotype(indiv, sigmas):

    # Initialize the individual (compute the inter matrix)
    indiv.evaluate(0, 0)

    nb_sigmas = len(sigmas)
    ext_phen = np.zeros((indiv.nb_genes, nb_sigmas))

    for i_sigma, sigma_env in enumerate(sigmas):
        temporal_expr = indiv.run_system(sigma_env)
        for i_gene, gene in enumerate(indiv.genes):
            ext_phen[i_gene, i_sigma] = temporal_expr[-1, i_gene]
    
    return ext_phen

# Random individual: influence of environmental supercoiling on final gene expression levels

In [ ]:
init_indiv = make_random_indiv(nb_genes=20, nb_mut=25)

In [ ]:
plot_expr(init_indiv, sigma_env=0, plot_title='', plot_name='random_gene_expr')

In [ ]:
init_indiv.already_evaluated=False
evotsc_plot.plot_expr_AB(init_indiv, sigma_A=sigma_A, sigma_B=sigma_B, color_by_type=False, plot_name='random_gene_expr_AB')

In [ ]:
importlib.reload(evotsc_plot)
evotsc_plot.plot_genome_and_tsc(init_indiv, sigma=0, color_by_type=False, print_ids=True, id_interval=1,
                                use_letters=True, show_bar=True, plot_name='random_genome_and_tsc.pdf')

In [ ]:
# See how gene activity levels depend on environmental supercoiling
def plot_activity_sigma_all_genes(indiv, plot_title=None, plot_name=None):

    colors = mpl.cm.get_cmap('viridis', indiv.nb_genes)(range(indiv.nb_genes))
    
    nb_sigmas = 1000
    
    plt.figure(figsize=(6, 4), dpi=dpi)
    plt.xlabel('Environment supercoiling')
    plt.ylabel('Final gene activity')
    plt.ylim(-0.05, 1.05)
    plt.grid(linestyle=':')
    if plot_title:
        plt.title(plot_title)

    sigmas = np.linspace(sigma_opt - 0.0125, sigma_opt + 0.1, nb_sigmas)
    ext_phen = compute_extended_phenotype(indiv, sigmas)

    for i_gene in range(indiv.nb_genes):
        linestyle = 'solid' if indiv.genes[i_gene].orientation == 0 else 'dashed'
        plt.plot(sigmas, ext_phen[i_gene, :],
                 linestyle=linestyle,
                 color=colors[i_gene],
                 label=f'Gene {i_gene}')
        
    # Plot what an isolated gene looks like
    final_exprs = np.exp(m * (1.0 / (1.0 + np.exp((indiv.sigma_basal + sigmas - indiv.sigma_opt)/indiv.epsilon)) - 1.0))

    plt.plot(sigmas, final_exprs, color='tab:red')
        
    #plt.legend(ncol=1)
                
    if plot_name:
        plt.savefig(plot_name + '.pdf', dpi=dpi, bbox_inches='tight')

In [ ]:
plot_activity_sigma_all_genes(init_indiv, plot_title='Random individual', plot_name='random_no_switch')

In [ ]:
def plot_all_single_switches(indiv):
    for i_gene in range(indiv.nb_genes):
        new_indiv = indiv.clone()
        new_indiv.already_evaluated = False
        # Switch one gene
        new_indiv.genes[i_gene].orientation = 1 - new_indiv.genes[i_gene].orientation
        plot_activity_sigma_all_genes(new_indiv,
                                      plot_title=f'Random individual switched gene {i_gene}',
                                      plot_name=f'random_switch_{i_gene}')

In [ ]:
plot_all_single_switches(init_indiv)

In [ ]:
def plot_phenotypic_distance(indiv):

    nb_sigmas = 50
    sigmas = np.linspace(-0.25, 0.15, nb_sigmas)

    indiv_phen = compute_extended_phenotype(indiv, sigmas)

    max_switches = 3

    all_switches = []

    # The distance of each individual with k gene switches from the original individual
    for nb_switches in range(1, max_switches+1):
        phen_dist = []

        for genes_to_switch in itertools.combinations(range(indiv.nb_genes), nb_switches):
            new_indiv = indiv.clone()
            new_indiv.already_evaluated = False

            # Switch genes
            for i_gene in genes_to_switch: 
                new_indiv.genes[i_gene].orientation = 1 - new_indiv.genes[i_gene].orientation

            new_indiv_phen = compute_extended_phenotype(new_indiv, sigmas)

            phen_dist.append(np.sqrt(np.square(indiv_phen - new_indiv_phen).sum()))

        all_switches.append(phen_dist)
            
    plt.xlabel('Number of switches')
    plt.xticks(range(1, max_switches+1))
    plt.ylabel('L2 distance')

    plt.violinplot(all_switches, showmeans=True)


In [ ]:
plot_phenotypic_distance(init_indiv)

## Dispersion of gene activity levels with mutations

In [ ]:
# Generate N mutants from an individual to see dispersion of gene activity levels
def plot_activity_mutation(indiv, sigma, mutation, plot_name=None):
    
    nb_mut = 1000

    activ = np.zeros(nb_mut)

    rng = np.random.default_rng()
    
    gene_positions, genome_size = indiv.compute_gene_positions(include_coding=False)

    for i_mut in range(nb_mut):
        


        # Generate a new mutant and evaluate it
        mut_indiv = indiv.clone()
        
        start_pos = rng.integers(0, genome_size)
        end_pos = rng.integers(0, genome_size)
        
        if end_pos < start_pos:
            start_pos, end_pos = end_pos, start_pos

        mut_indiv.perform_inversion(gene_positions, start_pos, end_pos)
        mut_indiv.already_evaluated = False
        
        (temporal_expr, _), _ = mut_indiv.evaluate(sigma, sigma)

        # Compute total gene activation levels
        activ[i_mut] = np.sum(np.square(temporal_expr[:, -1])) /  mut_indiv.nb_genes

    # Plot setup    
    plt.figure(figsize=(9, 3), dpi=dpi)
    plt.xlabel('Average squared gene activity')
    #plt.xlim(-0.025, 0.525)
    plt.ylabel('Number of mutants')
    plt.grid(linestyle=':')
    
    # Plot the histogram
    plt.hist(activ)
    
    # Plot the original activity
    (orig_expr, _), _ = indiv.evaluate(sigma, sigma)
    orig_activ = np.sum(np.square(temporal_expr[:, -1])) / indiv.nb_genes
    y_min, y_max = plt.ylim()
    plt.vlines(orig_activ, y_min, y_max, linestyle='--', linewidth=1,
                   color='tab:red', label='Original activity level')
    plt.ylim(y_min, y_max)
    
    plt.legend()

    plt.tight_layout()
    
    if plot_name:
        plt.savefig(plot_name + '.pdf', dpi=dpi, bbox_inches='tight')

In [ ]:
for i in range(1):
    plot_activity_mutation(make_random_indiv(nb_mut=100), sigma=sigma_A, mutation=evotsc.Mutation(inversion_poisson_lam=inversion_poisson_lam), plot_name=f'robustness_random_{i}')